In [1]:
import pandas as pd
from pathlib import Path
import sys
import numpy as np

# Notebook / root execution
ROOT = Path.cwd()


def find_date_column(df):
    candidates = ['date', 'datum', 'Datum', 'DATUM']
    for c in df.columns:
        if c.lower() in [x.lower() for x in candidates]:
            return c
    return None


def read_and_normalize(path: Path):
    df = pd.read_csv(path)
    date_col = find_date_column(df)
    if date_col is None:
        raise ValueError(f"No date column found in {path}")
    df[date_col] = pd.to_datetime(df[date_col], dayfirst=False, errors='coerce')
    df = df.rename(columns={date_col: 'date'})
    df = df.rename(columns={c: c.strip() for c in df.columns})
    return df


def find_column(df, keywords):
    kws = [k.lower() for k in keywords]
    for c in df.columns:
        lc = c.lower()
        for k in kws:
            if k in lc:
                return c
    return None


def standardize_sales_df(df):
    # id
    id_col = find_column(df, ['id'])
    if id_col and id_col != 'id':
        df = df.rename(columns={id_col: 'id'})

    # warengruppe
    war_col = find_column(df, ['wareng', 'warengruppe'])
    if war_col and war_col != 'warengruppe':
        df = df.rename(columns={war_col: 'warengruppe'})

    # umsatz
    u_col = find_column(df, ['umsatz'])
    if u_col:
        if u_col != 'umsatz':
            df = df.rename(columns={u_col: 'umsatz'})
        df['umsatz'] = pd.to_numeric(
            df['umsatz'].astype(str).str.replace(',', '.'),
            errors='coerce'
        )
    else:
        df['umsatz'] = pd.NA

    # ensure id exists
    if 'id' not in df.columns:
        df['id'] = pd.NA

    df = df.rename(columns={c: c.strip() for c in df.columns})
    return df


def pick_existing(*paths: Path) -> Path:
    for p in paths:
        if p.exists():
            return p
    raise FileNotFoundError(
        "None of these files exist:\n" + "\n".join(str(p) for p in paths)
    )


def wg6_is_available(d: pd.Timestamp) -> bool:
    # WG6 only exists in November and December
    if pd.isna(d):
        return False
    return d.month in [11, 12]



def main():
    # --- base inputs ---
    umsatz_path = ROOT / 'umsatzdaten_gekuerzt.csv'

    wetter_path = pick_existing(
        ROOT / 'wetter_imputed.csv',
        ROOT / 'wetter_expanded.csv',
        ROOT / 'wetter.csv',
    )

    kiwo_path = pick_existing(
        ROOT / 'kiwo_expanded.csv',
        ROOT / 'kiwo_clean_full_range.csv',
        ROOT / 'kiwo.csv',
    )

    school_path = pick_existing(
        ROOT / 'ferien_expanded.csv',
        ROOT / 'ferien_clean_full_range.csv',
        ROOT / 'Ferien_SH.csv',
    )

    public_path = pick_existing(
        ROOT / 'feiertage_expanded.csv',
        ROOT / 'feiertage_clean_full_range.csv',
        ROOT / 'Feiertage_holidays_sh_2013_2019.csv',
    )

    # test.csv sometimes in ROOT, sometimes in ROOT/analysis
    test_path = pick_existing(
        ROOT / 'test.csv',
        ROOT / 'analysis' / 'test.csv',
    ) if ((ROOT / 'test.csv').exists() or (ROOT / 'analysis' / 'test.csv').exists()) else None

    print('Reading base files...')
    umsatz = read_and_normalize(umsatz_path)
    wetter = read_and_normalize(wetter_path)
    kiwo = read_and_normalize(kiwo_path)
    school = read_and_normalize(school_path)
    public = read_and_normalize(public_path)

    # --- Standardize sales (+ optional continuation) ---
    umsatz = standardize_sales_df(umsatz)

    if test_path is not None and test_path.exists():
        print(f'Appending continuation file: {test_path}')
        test_df = read_and_normalize(test_path)
        test_df = standardize_sales_df(test_df)

        desired_cols = ['date', 'id', 'warengruppe', 'umsatz']
        for c in desired_cols:
            if c not in umsatz.columns:
                umsatz[c] = pd.NA
            if c not in test_df.columns:
                test_df[c] = pd.NA

        combined_sales = pd.concat(
            [umsatz[desired_cols], test_df[desired_cols]],
            ignore_index=True
        )

        combined_sales['date'] = pd.to_datetime(combined_sales['date'], errors='coerce')
        combined_sales['_id_sort'] = combined_sales['id'].astype(str).fillna('')
        combined_sales = (
            combined_sales
            .sort_values(by=['date', '_id_sort'], na_position='last')
            .drop(columns=['_id_sort'])
        )

        umsatz = combined_sales
        print(f'Combined sales rows: {len(umsatz)}')

    # Clean types for sales keys
    umsatz['date'] = pd.to_datetime(umsatz['date'], errors='coerce')
    umsatz['warengruppe'] = pd.to_numeric(umsatz['warengruppe'], errors='coerce')
    umsatz['id'] = pd.to_numeric(umsatz['id'], errors='coerce')

    # --- Normalize indicators ---
    school_ind = find_column(school, ['school_holiday', 'ferien', 'schulferien'])
    if school_ind is None or school_ind == 'date':
        school['school_holiday'] = 1
        school_ind = 'school_holiday'
    school = school[['date', school_ind]].rename(columns={school_ind: 'school_holiday'})

    public_ind = find_column(public, ['public_holiday', 'is_holiday', 'feiertag', 'holiday'])
    if public_ind is None or public_ind == 'date':
        public['public_holiday'] = 1
        public_ind = 'public_holiday'
    public = public[['date', public_ind]].rename(columns={public_ind: 'public_holiday'})

    kiwo_ind = find_column(kiwo, ['kielerwoche', 'kiwo'])
    if kiwo_ind and kiwo_ind not in ['KielerWoche', 'date']:
        kiwo = kiwo.rename(columns={kiwo_ind: 'KielerWoche'})

    # --- Build full date calendar for exogenous features ---
    start_date = min(
        wetter['date'].min(),
        kiwo['date'].min(),
        school['date'].min(),
        public['date'].min(),
        umsatz['date'].min(),
    )
    end_date = max(
        wetter['date'].max(),
        kiwo['date'].max(),
        school['date'].max(),
        public['date'].max(),
    )

    calendar = pd.DataFrame({'date': pd.date_range(start=start_date, end=end_date, freq='D')})

    base = (
        calendar
        .merge(wetter, on='date', how='left')
        .merge(kiwo, on='date', how='left')
        .merge(school, on='date', how='left')
        .merge(public, on='date', how='left')
    )

    # Force calendar flags to 0/1 Int64
    for c in ['KielerWoche', 'school_holiday', 'public_holiday']:
        if c in base.columns:
            base[c] = pd.to_numeric(base[c], errors='coerce').fillna(0).astype('Int64')
            base[c] = (base[c] > 0).astype('Int64')

    # Weather ints
    for c in ['Bewoelkung', 'Windgeschwindigkeit']:
        if c in base.columns:
            base[c] = pd.to_numeric(base[c], errors='coerce').round(0).astype('Int64')

    # --- Build FULL (date × warengruppe) grid, with wg6 seasonal ---
    warengruppen_all = [1, 2, 3, 4, 5]  # always
    # build per-date grid
    rows = []
    for d in base['date']:
        # always include 1..5
        for wg in warengruppen_all:
            rows.append((d, wg))
        # include wg6 only in season
        if wg6_is_available(d):
            rows.append((d, 6))

    grid = pd.DataFrame(rows, columns=['date', 'warengruppe'])

    # Generate Kaggle-style id: yymmdd + warengruppe (as last digit)
    grid['id'] = grid['date'].dt.strftime('%y%m%d').astype(int) * 10 + grid['warengruppe'].astype(int)

    # --- Merge sales onto grid ---
    sales_keys = umsatz[['date', 'warengruppe', 'id', 'umsatz']].copy()
    # Keep only valid key rows
    sales_keys = sales_keys.dropna(subset=['date', 'warengruppe', 'id'])

    merged = grid.merge(sales_keys, on=['date', 'warengruppe', 'id'], how='left')

    # --- Merge exogenous features onto every row ---
    merged = merged.merge(base, on='date', how='left')

    # ---------------------------------------------------------
    # DIAGNOSTIC: list dates with missing sales in TRAIN RANGE
    # (now correctly defined as: umsatz missing even though the row exists)
    # ---------------------------------------------------------
    start_diag = pd.to_datetime("2013-07-01")
    end_diag   = pd.to_datetime("2017-07-31")

    missing_sales_dates = (
        merged[
            (merged["date"] >= start_diag) &
            (merged["date"] <= end_diag) &
            (merged["umsatz"].isna())
        ][["date", "public_holiday", "school_holiday"]]
        .drop_duplicates()
        .sort_values("date")
    )

    print("\nDates with missing sales (unique):")
    print(missing_sales_dates)

    print("\nMissing sales summary:")
    print("Total dates:", len(missing_sales_dates))
    print("Public holiday counts:")
    print(missing_sales_dates["public_holiday"].value_counts(dropna=False))
    print("School holiday counts:")
    print(missing_sales_dates["school_holiday"].value_counts(dropna=False))

    missing_sales_dates.to_csv("missing_sales_dates.csv", index=False)
    print("\nSaved: missing_sales_dates.csv")

    # --- Final sorting/column order ---
    merged = merged.sort_values(by=['date', 'warengruppe', 'id'], na_position='last')

    preferred = [
        'date', 'warengruppe', 'id', 'umsatz',
        'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode',
        'KielerWoche', 'school_holiday', 'public_holiday'
    ]
    merged = merged[[c for c in preferred if c in merged.columns] +
                    [c for c in merged.columns if c not in preferred]]

    # --- Make umsatz blank (empty cell) instead of NaN in CSV ---
    # (only affects output formatting; reading back will interpret blanks as NaN unless you set keep_default_na=False)
    merged['umsatz'] = merged['umsatz'].apply(lambda x: '' if pd.isna(x) else x)

    out_path = ROOT / 'merged_expanded_data_fullcalendar.csv'
    merged.to_csv(out_path, index=False, na_rep='NaN')

    print(f'\nWrote merged CSV to: {out_path}')
    print("Merged date range:", merged['date'].min(), "→", merged['date'].max())
    # last date with observed sales (not blank)
    last_sales = merged.loc[merged['umsatz'].astype(str).str.len() > 0, 'date'].max()
    print("Last non-empty umsatz date:", last_sales)


if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print('Error during merge:', e, file=sys.stderr)
        raise


Reading base files...
Appending continuation file: /workspaces/Group-12---Introduction-to-Data-Science-Machine-Learning/analysis/test.csv
Combined sales rows: 11164


/tmp/ipykernel_125932/3124034061.py:143: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_sales = pd.concat(



Dates with missing sales (unique):
            date  public_holiday  school_holiday
2854  2013-07-12               0               1
2879  2013-07-17               0               1
2929  2013-07-27               0               1
2934  2013-07-28               0               1
2939  2013-07-29               0               1
...          ...             ...             ...
9439  2016-12-31               0               1
9440  2017-01-01               1               0
9955  2017-04-14               1               1
10040 2017-05-01               1               0
10360 2017-07-04               0               0

[95 rows x 3 columns]

Missing sales summary:
Total dates: 95
Public holiday counts:
public_holiday
0    76
1    19
Name: count, dtype: Int64
School holiday counts:
school_holiday
0    48
1    47
Name: count, dtype: Int64

Saved: missing_sales_dates.csv

Wrote merged CSV to: /workspaces/Group-12---Introduction-to-Data-Science-Machine-Learning/merged_expanded_data_fullcalen